In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.losses import *
from keras.applications.resnet50 import ResNet50
from keras.utils.vis_utils import plot_model
from models import *
from keras.optimizers import adam

from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import mnist
from keras.utils import to_categorical
import random

Using TensorFlow backend.


In [2]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
img =  Image.open("/Users/akiyamatakumi/Desktop/project/AI_comp/lemon_classification/notebooks/test_0003.jpg")
img_resize = np.array(img.resize((79,79)))
img_resize[:,:,0][np.newaxis,...].shape

x_train = np.tile(img_resize[:,:, 0][np.newaxis,...], (22,1,1))
x_train = np.concatenate((x_train, img_resize[:,:,2][np.newaxis,...]))
print(x_train.shape)
x_train = np.tile(x_train[np.newaxis, ...], (16,1,1,1))
print(x_train.shape)

x_train = x_train[..., np.newaxis]
x_train = np.concatenate((x_train, x_train), axis = 4)
img = np.transpose(x_train, (0,1,4,2,3))
print(x_train.shape)
plt.imshow(img[0][22][1])

y_train = np.zeros(16).reshape(16,1)

(23, 79, 79)
(16, 23, 79, 79)
(16, 23, 79, 79, 2)


In [3]:
x_train.shape
# x_train = np.transpose(x_train,(0,4,2,3,1))
# x_train.shape

(16, 23, 79, 79, 2)

In [4]:
batch_size = 1

In [9]:
# we create two instances with the same arguments
data_gen_args = dict(featurewise_center=True,
                     featurewise_std_normalization=True,
                     rotation_range=90.,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.2)
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

def gen_aug_chunk(in_gen, y, batch_size):
    cnt = 0
    for i, x in enumerate(in_gen):
        print(i)
        x_img = x[np.newaxis, ...]
#         print(x_img.shape)
        x_img_0 = x_img.swapaxes(1, 4)[:, 0]
        x_img_1 = x_img.swapaxes(1, 4)[:, 1]
#         print(x_img_0.shape)
        seed = random.randint(0, 100)
        image_generator = image_datagen.flow((x_img_0, y[i]), shuffle=True, seed=seed, batch_size = batch_size)
        mask_generator = mask_datagen.flow((x_img_1, y[i]), shuffle=True, seed=seed, batch_size = batch_size)
        x_scat_0 = next(image_generator)[0]
        x_scat_1 = next(mask_generator)[0]
#         print(len(image_generator))
        x_scat = np.concatenate((x_scat_0,x_scat_1), axis = 0)
        x_scat = np.expand_dims(x_scat,0).swapaxes(1, 4)
        print(x_scat.shape)

        train_generator = zip(image_generator, mask_generator)
#         print("cnt:" + str(cnt))
        cnt+=1
        yield x_scat, y[i]
        

In [10]:
train_aug_gen = gen_aug_chunk(x_train, y_train, batch_size = 5)
# type(train_aug_gen)
x_out, y_out = next(train_aug_gen)
# print(x_out.shape)

0
(1, 23, 79, 79, 2)


In [11]:
from keras.layers import ConvLSTM2D, Bidirectional, BatchNormalization, Conv3D, Cropping3D, ZeroPadding3D, Activation, Input
from keras.layers import MaxPooling3D, UpSampling3D, Deconvolution3D, concatenate, Flatten, Dense
from keras.models import Model
import keras

in_layer = Input((23, 79, 79, 2))
bn = BatchNormalization()(in_layer)
cn1 = Conv3D(8, 
             kernel_size = (1, 5, 5), 
             padding = 'same',
             activation = 'relu')(bn)
fl = Flatten()(cn1)
out = Dense(1, activation = "sigmoid")(fl)
sim_model = Model(inputs = [in_layer], outputs = [out])
sim_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 23, 79, 79, 2)     0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 23, 79, 79, 2)     8         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 23, 79, 79, 8)     408       
_________________________________________________________________
flatten_1 (Flatten)          (None, 1148344)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1148345   
Total params: 1,148,761
Trainable params: 1,148,757
Non-trainable params: 4
_________________________________________________________________


In [12]:
sim_model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['binary_accuracy', 'mse'])
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
weight_path="{}_weights.best.hdf5".format('convlstm_model')

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=5, verbose=1, mode='auto', epsilon=0.0001, cooldown=5, min_lr=0.0001)
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=10) # probably needs to be more patient, but kaggle time is limited
callbacks_list = [checkpoint, early, reduceLROnPlat]

/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [1]:
sim_model.fit_generator(train_aug_gen, 
                        epochs=20,
                        steps_per_epoch = 100)

NameError: name 'sim_model' is not defined